This Jupiter notebook is used to download, pre-process and save data about all test data in the Czech Republic from 
the webpage of the Czech...... Output from this notebook is CSV file with downloaded data about........ that will be further processed, cleaned and aggregated.


os package is used only to check whether we are in the correct working directory.

In the def, we will be applying methods from requests (downloading site content), BeautifulSoup (machine reading of downloaded data), tqdm (interactive measurement of the downloading progress) and time (setting pause between requests from the site) packages.

After the data are downloaded we use pandas to pre-process and save them.

After downloading the packages, I will check the data structure in order to make sure that we can scrape the given information 
here below we used BeutfulSoup library to request the data.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from tqdm import tqdm
import re
import numpy as np
html_text = requests.get("https://testovani.uzis.cz/").text.strip()
soup = BeautifulSoup(html_text, 'lxml')


In [20]:
for a in soup.find_all('a', class_ = "list_link", href=True):
    print ("Found the URL:", a['href'])

In [49]:
# Getting an unique identifier of each place - the id seems to be stored in href of <a> tag right after "?id="
# All obtained Ids are then assigned to TestCenterIds array

testingCenterHyperlinkTags = soup.find_all('a', class_ = "list__link", href=True)

TestCenterIds = []

for testingCenterHyperlinkTag in testingCenterHyperlinkTags[:5]:
    testingCenterHyperlink = testingCenterHyperlinkTag['href'][11:47]
    print ("Found the id:", testingCenterHyperlink)
    TestCenterIds.append(testingCenterHyperlink)

TestCenterIds[0:5]


Found the id: a81a99e2-90bd-494c-ae42-7090f93115ca
Found the id: 42c85d23-21e4-4fe9-baef-9ab06c0cbe6d
Found the id: 4bf34117-aadf-400d-91ec-77dc8633874f
Found the id: e3bc023c-27a1-496f-8a5c-665b0e103e60
Found the id: 4e6f8c5e-1b70-4289-a2ab-ae9065304eb5


['a81a99e2-90bd-494c-ae42-7090f93115ca',
 '42c85d23-21e4-4fe9-baef-9ab06c0cbe6d',
 '4bf34117-aadf-400d-91ec-77dc8633874f',
 'e3bc023c-27a1-496f-8a5c-665b0e103e60',
 '4e6f8c5e-1b70-4289-a2ab-ae9065304eb5']

The OOP structure should consist of following objects: Place, DateCapacity, Region. TestingPlace object is the main object in our project and in order to have all data consistent and in order, we will work with TestingPlace ID as an unique identifier instead of the Name.

In [50]:
# Defince Place class

class Place:
    
    def __init__(self, id, URL_address, region):
        self.id = id
        self.URL_address = URL_address
        self.region = region
    
    def basics(self):
        return f"TestingCapacity record with id {self.id} can be found at {self.URL_address}"

In [ ]:
# Get all places the then for each Test Center Id try to find Place record with matchind Id. 
# If no such record found, create new Place record.

# Query should never be inside the loop


for PlaceId in TestCenterIds:
    pass

In [2]:
test_centers = soup.find_all('div', class_="list__result")
header = soup.find('div', class_="list__headline").text.strip()
print(f" Header: {header}") 

for test_center in test_centers:
    company_name = test_center.find('span', class_="list__col").text.replace('\n ', '')
    #callendar= test_center.find('div', class_="main")#.text.replace('\n ', '')
    print(f" Company Name : {company_name.strip()}")
   
    print(' ')
    


C Laboratoře Praha
 
 Company Name : EUC Laboratoře s.r.o. - odběrové místo Praha Bohdalec
 
 Company Name : EUC Laboratoře s.r.o. Horní Počernice - odběrové místo
 
 Company Name : Fakultní nemocnice Královské Vinohrady
 
 Company Name : Fakultní nemocnice v Motole
 
 Company Name : Forenzní DNA servis s.r.o. - odběrové místo FN Bulovka
 
 Company Name : GenDetective - Černý Most
 
 Company Name : GenDetective - Čestlice
 
 Company Name : GenDetective s.r.o. - odběrové místo Praha Kutnohorská
 
 Company Name : GenDetective s.r.o. - odběrové místo Praha U Technoplynu
 
 Company Name : GenDetective s.r.o.- odběrové místo SAPA
 
 Company Name : GenDetective s.r.o.-odběrové místo Drive-In Braník
 
 Company Name : GenDetective s.r.o.-odběrové místo Drive-In Zličín
 
 Company Name : GenDetective s.r.o.-odběrové místo Hamr Záběhlice
 
 Company Name : GenDetective s.r.o.-odběrové místo Karlovy lázně
 
 Company Name : Genetická laboratoř sanatoria Pronatal s.r.o. - odběrové místo Zelený pruh
 

In [3]:
header = soup.find('div', class_="list__headline").text.strip()
header# in order to check how many free places are available

'Hlavní město Praha (Počet OM: 102 - V následujících 3 dnech volná/celková kapacita: 18662 / 24179)'

Getting info about the website

In [4]:
import pandas as pd # to be saved and aggreagated into datafame
import numpy as np

In [5]:
#Getting tqdm in order to show a progress bar
from tqdm import tqdm_notebook as tqdm

In [6]:
def get_page_links(url):# Firstly created the function gets the data and returns the links of the each company 
    sleep(0.2) 
    base_url='https://testovani.uzis.cz/'
    r = requests.get(url)
    soup= BeautifulSoup(r.text,'lxml')
    links=soup.select('div.list__result a')
    return[ base_url + link.attrs['href'] for link in links] 

In [7]:
    
def test_locatio_detail(url): # Created the second function which grabs all necessary information form each comapny and returns
    r = requests.get(url)
    soup= BeautifulSoup(r.text,'lxml')
    company_details={
        'Name':soup.select_one('div.detail__main h1').text.strip().replace('\n',' '),
        'Adress':soup.select_one('div.info td:nth-child(2)').text.strip().replace('\n',''),
        'Upřesnění polohy':soup.select_one('div.info :nth-child(2) td:nth-child(2)').text.strip().replace('\n',''),
        'Telefon':soup.select_one('div.info a').text.strip().replace('\n',''),
        'Poznámka':soup.select_one('div.info :nth-child(4) td:nth-child(2)').text.strip().replace('\n',''),
        'Příjem':soup.select_one('div.info :nth-child(5) td:nth-child(2)').text.strip().replace('\n',''),
        'Cena za samoodběř':soup.select_one('div.info :nth-child(6) td:nth-child(2)').text.strip().replace('\n',''),
        'Způsob rezervace':soup.select_one('div.info :nth-child(7) td:nth-child(2)').text.strip().replace('\n',''),
        'Odkaz na rezervační systém':soup.select_one('div.info :nth-child(8) td:nth-child(2)').text.strip().replace('\n',''),
        'DRIVE-IN':soup.select_one('div.info :nth-child(9) td:nth-child(2)').text.strip().replace('\n',''),
        #'Av_days':pd.DataFrame([soup.select_one('div.opening').text.strip().replace('\n','')])
   
    }
    #print(company_details)    
    return company_details
#test_location_detail('https://testovani.uzis.cz//Detail?id=b84211a4-9c9e-4773-8439-71bb18fde2d4&backURL')  

In [16]:

#get_page_links("https://testovani.uzis.cz/")    

def main():# Created 3rd function which focuses on gattering all the information into one dictionary, to be saved in main function
   results=[]
   urls= get_page_links('https://testovani.uzis.cz/')
   comp_info=[test_locatio_detail(url) for url in tqdm(urls)]
   results.append(comp_info) #creating the data storage for further use, as we assume to use the callendar sting
   return results

   print(results)


In [ ]:
print(main()) #Just for control, otherwise "messy" JSON taking up too much space

In [17]:
def test_locatio_detail(url): # Created the second function which grabs all necessary information form each company and returns
    r = requests.get(url)
    soup= BeautifulSoup(r.text,'lxml')
    company_details={
        'Name':soup.select_one('div.detail__main h1').text.strip().replace('\n',' '),
        'Address':soup.select_one('div.info td:nth-child(2)').text.strip().replace('\n',''),
        'Upřesnění polohy':soup.select_one('div.info :nth-child(2) td:nth-child(2)').text.strip().replace('\n',''),
        'Telefon':soup.select_one('div.info a').text.strip().replace('\n',''),
        'Poznámka':soup.select_one('div.info :nth-child(4) td:nth-child(2)').text.strip().replace('\n',''),
        'Příjem':soup.select_one('div.info :nth-child(5) td:nth-child(2)').text.strip().replace('\n',''),
        'Cena za samoodběr':soup.select_one('div.info :nth-child(6) td:nth-child(2)').text.strip().replace('\n',''),
        'Způsob rezervace':soup.select_one('div.info :nth-child(7) td:nth-child(2)').text.strip().replace('\n',''),
        'Odkaz na rezervační systém':soup.select_one('div.info :nth-child(8) td:nth-child(2)').text.strip().replace('\n',''),
        'DRIVE-IN':soup.select_one('div.info :nth-child(9) td:nth-child(2)').text.strip().replace('\n',''),
        #'Av_days':pd.DataFrame([soup.select_one('div.opening').text.strip().replace('\n','')])
   
    }
    #print(company_details)    
    return company_details
f=test_locatio_detail('https://testovani.uzis.cz//Detail?id=b84211a4-9c9e-4773-8439-71bb18fde2d4&backURL')
pd.DataFrame(f,index=['1',])

,Name,Address,Upřesnění polohy,Telefon,Poznámka,Příjem,Cena za samoodběr,Způsob rezervace,Odkaz na rezervační systém,DRIVE-IN
1,"Vsetínská nemocnice, a.s., odběrové místo","Nemocniční 955, 755 01 Vsetín",infekční ambulance,571818384,PCR testy - odběrové centrum - Infekční ambula...,Osoby se žádánkou (od lékaře/KHS),,online webová aplikace,nemocnice-vs.cz,Ne


Cleaned and used the previous main function to set structured data.

In [18]:
def main():# Created 3rd function which focuses on gattering all the information into one dictionary, to be saved in main function
   #results=[]
   urls= get_page_links('https://testovani.uzis.cz/')
   comp_info=pd.DataFrame([test_locatio_detail(url) for url in tqdm(urls)])
   #results.append(comp_info)
   return comp_info

main()

<ipython-input-18-29c69cb3d68c>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  comp_info=pd.DataFrame([test_locatio_detail(url) for url in tqdm(urls)])


,Name,Address,Upřesnění polohy,Telefon,Poznámka,Příjem,Cena za samoodběr,Způsob rezervace,Odkaz na rezervační systém,DRIVE-IN
0,#NaŽádanku.cz – Odběrové centrum Praha 15 (H. ...,"Park na křižovatce ulic Ravennská a Milánská, ...",Zelené oplocení v parku na křižovatce ulic Rav...,+420774605535,Provádíme jak Antigenní testy jednou za 7 dnů ...,Osoby se žádánkou (od lékaře/KHS),,Přes webové stránky na přesný čas,www.nazadanku.cz,Ne
1,#Vyšetři.mě – Odběrové centrum Praha 15 (H. Mě...,"Park na křižovatce ulic Ravennská a Milánská, ...",Zelené oplocení v parku na křižovatce ulic Rav...,+420771156107,Rezervaci je nutno provést na webových stránká...,Osoby se žádankou i samoplátce,,Přes webové stránky,www.vysetri.me,Ne
2,#Vyšetři.mě – Odběrové centrum Praha 6 (Ruzyně),Drnovská 1112/60 161 00 Praha 6 - Ruzyně,Příjezdová cesta zezadu budovy (naproti adrese...,+420771156107,Rezervaci je nutno provést na webových stránká...,Osoby se žádankou i samoplátce,,Přes webové stránky,www.vysetri.me,Ano
3,AESKULAB - odběrové místo VIVO Hostivař,"Praha 10, Švehlova 1391/32","Odběrové místo - OC VIVO Hostivař, parkoviště",+420800737383,,Osoby se žádankou i samoplátce,,www.rezervace.aeskulab.cz,rezervace.aeskulab.cz,Ne
4,"AESKULAB Praha 6 - OM poliklinika DAM, Brixiho","Brixiho 1968/1, 162 00 Praha 6 - Břevnov","Poliklinika DAM, samostatný vchod vpravo z Bri...",800737383,,Osoby se žádankou i samoplátce,,Přes webové stránky,rezervace.aeskulab.cz,Ne
...,...,...,...,...,...,...,...,...,...,...
346,Mobilní zdravotní sestra - odběrové místo Vala...,"Zdeňka Fibicha 287, 757 01 Valašské Meziříčí",,,,,,telefonicky,,Ne
347,Nemocnice Valašské Meziříčí - odběrové místo,"U Nemocnice 980, 757 01, Valašské Meziříčí",odběrové centrum - poblíž vrátnice,571758605,ODBĚROVÉ CENTRUM (OC) - nachází se vedle antig...,Osoby se žádankou i samoplátce,,Pro samoplátce – on-line,www.agellab.cz,Ne
348,"Uherskohradišťská nemocnice a. s., odběrové místo","J. E. Purkyně 365, 686 06 Uherské Hradiště",areál nemocnice,724631022,"PCR - testy: nákladní brána, v části budovy A-...",Osoby se žádankou i samoplátce,,Přes webové stránky,ambulance.nemuh.cz,Ne
349,"Vaše laboratoře s.r.o., Zlín - odběrové místo",Tř. T. Bati 3910,po levé straně budovy,739408931,,Osoby se žádánkou (od lékaře/KHS),,Reservatic,reservatic.com,Ne


In [19]:
df=main()

<ipython-input-18-29c69cb3d68c>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  comp_info=pd.DataFrame([test_locatio_detail(url) for url in tqdm(urls)])


In [130]:
df.to_csv('Covid.csv')

We will next load the data to our working directory to process the data clean and put it in commercially friendly way

In [133]:
data = pd.read_csv('Covid.csv', index_col = 0)
data.head(10)

,Name,Adress,Upřesnění polohy,Telefon,Poznámka,Příjem,Cena za samoodběř,Způsob rezervace,Odkaz na rezervační systém,DRIVE-IN
0,#NaŽádanku.cz – Odběrové centrum Praha 15 (H. ...,"Park na křižovatce ulic Ravennská a Milánská, ...",Zelené oplocení v parku na křižovatce ulic Rav...,4.207746e+11,NaN,Osoby se žádánkou (od lékaře/KHS),NaN,Přes webové stránky na přesný čas,www.nazadanku.cz,Ne
1,#NaŽádanku.cz – Odběrové centrum Praha 6 (Ruzyně),Drnovská 1112/60 161 00 Praha 6 - Ruzyně,Příjezdová cesta zezadu budovy (naproti adrese...,4.207746e+11,NaN,Osoby se žádánkou (od lékaře/KHS),NaN,Přes webové stránky na přesný čas,www.nazadanku.cz,Ano
2,#Vyšetři.mě – Odběrové centrum Praha 15 (H. Mě...,"Park na křižovatce ulic Ravennská a Milánská, ...",Zelené oplocení v parku na křižovatce ulic Rav...,4.207712e+11,POZOR! Nově nás najdete v parku na křižovatce ...,Samoplátce,NaN,Přes webové stránky,www.vysetri.me,Ne
3,#Vyšetři.mě – Odběrové centrum Praha 6 (Ruzyně),Drnovská 1112/60 161 00 Praha 6 - Ruzyně,Příjezdová cesta zezadu budovy (naproti adrese...,4.207712e+11,Aktuálně testujeme pouze na přímý výskyt labor...,Samoplátce,NaN,Přes webové stránky,www.vysetri.me,Ano
4,AESKULAB - odběrové místo VIVO Hostivař,"Praha 10, Švehlova 1391/32","Odběrové místo - OC VIVO Hostivař, parkoviště",NaN,NaN,Osoby se žádankou i samoplátce,NaN,www.rezervace.aeskulab.cz,www.rezervace.aeskulab.cz,Ne
5,AESKULAB Praha 8 - Mazurská,"Mazurská 484/2, 181 00 Praha 8 - Troja",Parkoviště polikliniky Mazurská,8.007374e+08,NaN,Osoby se žádankou i samoplátce,NaN,Přes webové stránky,rezervace.aeskulab.cz,Ne
6,AESKULAB Praha Čs.exilu,"Čs.exilu 2175/36, Praha 12",NaN,4.208007e+11,NaN,Osoby se žádankou i samoplátce,NaN,Elektronický rezervační systém,rezervace.aeskulab.cz,Ne
7,AESKULAB Praha Vajgarská,"Vajgarská 1141, Praha 9",NaN,4.208007e+11,Na místě možno platit kartou nebo hotově.,Osoby se žádankou i samoplátce,NaN,Elektronický rezervační systém,rezervace.aeskulab.cz,Ne
8,"AKESO holding - OM Chlumecká, Praha 9 - Černý ...","Chlumecká 2420, Praha 9, Černý Most, 198 29",Parkoviště u vstupu do Sconto nábytek,4.207251e+11,Odběr není možný bez předchozího objednání!\rR...,Samoplátce,NaN,Online rezervační portál nebo telefonní linka,www.antigenni-testy-covid.cz,Ne
9,"AKESO holding - OM Jeremiášova, Praha 5 - Stod...","Jeremiášova 947/16, 155 00 Praha 5 - Stodůlky",Parkoviště u vstupu do Sconto nábytek,4.207251e+11,Odběr není možný bez předchozího objednání!\rR...,Samoplátce,NaN,Online rezervační portál nebo telefonní linka,www.antigenni-testy-covid.cz,Ne


I have tried to scrap the calendar, saved in csv file.

In [20]:
def calnedar(url): # Created the second function which grabs all necessary information form each comapny and returns
    r = requests.get(url)
    soup= BeautifulSoup(r.text,'lxml')
    company_calendars={
        'Calendar':soup.select_one('div.slots__list').text.strip().replace('\n',' ')
    }
    
    return company_calendars
Ca=calnedar('https://testovani.uzis.cz/Detail?id=f4d02b50-1561-4c7e-9cf4-8d85b2d985ea&backURL=/')
Ca

{'Calendar': 'Po 21.6.   Maximální kapacita     Út 22.6.   Volná kapacita 45 z 56     St 23.6.   Volná kapacita 30 z 56     Čt 24.6.   Volná kapacita 33 z 56     Pá 25.6.   Volná kapacita 42 z 56     So 26.6.   Volná kapacita 42 z 58     Ne 27.6.   Volná kapacita 36 z 58'}

In [67]:
def all_clendarInfo():# Here I will source all available calendar info.
   #results=[]
   urls= get_page_links('https://testovani.uzis.cz/')
   calnedars=pd.DataFrame([calnedar(url) for url in urls])
   #results.append(comp_info)
   return calnedars

all_clendarInfo()

,Calendar
0,St 5.5. Maximální kapacita Čt 6.5. Vol...
1,St 5.5. Maximální kapacita Čt 6.5. Vol...
2,St 5.5. Maximální kapacita Čt 6.5. Vol...
3,St 5.5. Maximální kapacita Čt 6.5. Vol...
4,St 5.5. Žádná data Čt 6.5. Žádná data ...
...,...
317,St 5.5. Maximální kapacita Čt 6.5. Zav...
318,St 5.5. Žádná data Čt 6.5. Žádná data ...
319,St 5.5. Žádná data Čt 6.5. Žádná data ...
320,St 5.5. Maximální kapacita Čt 6.5. Vol...


In [ ]:
#Getting capacities info from each calendar line


In [38]:
cad_dataframe=all_clendarInfo()
cad_dataframe.to_csv('calendar.csv')